Prepare for Live Paper Trading

In [1]:
import os
import alpaca_trade_api as tradeapi
import numpy as np
import pandas as pd
import yfinance as yf

# 1. Setup API Connection
API_KEY = os.environ.get('APCA_API_KEY_ID')
API_SECRET = os.environ.get('APCA_API_SECRET_KEY')
BASE_URL = 'https://paper-api.alpaca.markets'  # Fixed: removed extra space

api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL)

# 2. Define Parameters from your research
STRATEGY_PARAMS = {'rsi_entry': 30, 'atr_multiplier': 1.5}

def calculate_features(df):
    """Engineers all necessary features from the raw OHLCV data."""
    df = df.copy()
    
    # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
    delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    
    # Handle division by zero and NaN values
    rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # 2. Bollinger Bands
    df['sma_20'] = df['spy_close'].rolling(window=20).mean()
    df['std_20'] = df['spy_close'].rolling(window=20).std()
    df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
    df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

    # 3. Rolling Z-Score
    df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

    # 4. Average True Range (ATR) - Fixed column references
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
    low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr'] = tr.rolling(window=14).mean()
    
    return df

def get_market_data(start_date="2005-01-01"):
    """Downloads SPY and VIX data and engineers features."""
    spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
    vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

    # Reset any potential multi-level indexes and ensure clean column structure
    if isinstance(spy_data.columns, pd.MultiIndex):
        spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
    if isinstance(vix_data.columns, pd.MultiIndex):
        vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
    
    # Select only the columns we need
    spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
    vix_data = vix_data[['Close']].copy()
    vix_data = vix_data.rename(columns={'Close': 'vix'})
    
    # Align the date indices properly
    common_dates = spy_data.index.intersection(vix_data.index)
    spy_data = spy_data.loc[common_dates]
    vix_data = vix_data.loc[common_dates]
    
    # Combine into a single DataFrame using merge
    df = spy_data.copy()
    df['vix'] = vix_data['vix']
    df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

    # Calculate features using the function
    df = calculate_features(df)
    
    # Calculate daily returns for the backtest
    df['daily_return'] = df['spy_close'].pct_change()

    return df.dropna()

def generate_vol_norm_rsi_signals(df, rsi_entry=30, rsi_exit=50, atr_multiplier=1.0, vix_threshold=40):
    """
    Generates trading signals based on the Volatility-Normalised RSI strategy.
    Returns a DataFrame with a 'signal' column (1 for long, -1 for short, 0 for flat).
    """
    signals = pd.DataFrame(index=df.index)
    signals['signal'] = 0

    # Entry Conditions
    long_entry_condition = (df['rsi'] < rsi_entry) & \
                           (df['spy_close'] < (df['spy_close'].shift(1) - (atr_multiplier * df['atr'])))
    
    short_entry_condition = (df['rsi'] > (100 - rsi_entry)) & \
                            (df['spy_close'] > (df['spy_close'].shift(1) + (atr_multiplier * df['atr'])))

    # Exit Conditions
    long_exit_condition = df['rsi'] > rsi_exit
    short_exit_condition = df['rsi'] < (100 - rsi_exit)

    # State machine to hold positions
    position = 0
    for i in range(len(df)):
        if position == 0: # If flat
            if long_entry_condition.iloc[i]:
                position = 1
            elif short_entry_condition.iloc[i]:
                position = -1
        elif position == 1: # If long
            if long_exit_condition.iloc[i]:
                position = 0
        elif position == -1: # If short
            if short_exit_condition.iloc[i]:
                position = 0
        signals['signal'].iloc[i] = position

    # VIX Regime Filter - Fixed the assignment
    vix_high_mask = df['vix'] > vix_threshold
    signals.loc[vix_high_mask, 'signal'] = 0
    
    return signals['signal']

def get_current_position():
    """Gets current SPY position size from Alpaca."""
    try:
        position = api.get_position('SPY')
        return int(float(position.qty))  # Convert to int after float
    except Exception as e:
        # API returns 404 if no position, handle it gracefully
        print(f"No position found: {e}")
        return 0

def reconcile_position():
    """Main execution function to align live position with strategy signal."""
    # 1. Fetch latest data
    # Note: For live, you'd use Alpaca's market data API for up-to-the-minute data
    # Using yfinance here for consistency with the research environment.
    print("Fetching latest market data...")
    market_data = get_market_data() # Gets all historical data up to yesterday
    
    # 2. Generate target signal for today
    signals = generate_vol_norm_rsi_signals(market_data, **STRATEGY_PARAMS)
    target_signal = signals.iloc[-1] # -1 for short, 1 for long, 0 for flat
    
    print(f"Target signal is: {target_signal}")
    
    # 3. Get current live position
    current_qty = get_current_position()
    current_signal = np.sign(current_qty) if current_qty != 0 else 0
    
    print(f"Current position is: {current_qty} shares (Signal: {current_signal})")
    
    # 4. Compare and execute
    if target_signal == current_signal:
        print("Position is already aligned. No action needed.")
        return

    # --- Execution Logic ---
    if target_signal == 0: # Target is flat
        print("Target is flat. Closing existing position.")
        try:
            api.close_position('SPY')
            print("Position closed successfully.")
        except Exception as e:
            print(f"Error closing position: {e}")
    elif target_signal == 1: # Target is long
        if current_signal == -1: # If currently short, close and then open long
            print("Closing short position to go long...")
            try:
                api.close_position('SPY')
            except Exception as e:
                print(f"Error closing short position: {e}")
        
        # Sizing logic (simplified here, use your Volatility Parity model)
        try:
            account = api.get_account()
            account_equity = float(account.equity)
            target_position_size_usd = account_equity * 0.20 # Example: 20% of equity
            latest_price = market_data['spy_close'].iloc[-1]
            qty_to_buy = int(target_position_size_usd / latest_price)
            
            print(f"Submitting buy order for {qty_to_buy} shares.")
            api.submit_order(
                symbol='SPY', 
                qty=qty_to_buy, 
                side='buy', 
                type='market', 
                time_in_force='day'
            )
            print("Buy order submitted successfully.")
        except Exception as e:
            print(f"Error submitting buy order: {e}")
        
    elif target_signal == -1: # Target is short
        if current_signal == 1: # If currently long, close first
            print("Closing long position to go short...")
            try:
                api.close_position('SPY')
            except Exception as e:
                print(f"Error closing long position: {e}")
        
        # Sizing logic
        try:
            account = api.get_account()
            account_equity = float(account.equity)
            target_position_size_usd = account_equity * 0.20
            latest_price = market_data['spy_close'].iloc[-1]
            qty_to_short = int(target_position_size_usd / latest_price)
            
            print(f"Submitting sell (short) order for {qty_to_short} shares.")
            api.submit_order(
                symbol='SPY', 
                qty=qty_to_short, 
                side='sell', 
                type='market', 
                time_in_force='day'
            )
            print("Sell order submitted successfully.")
        except Exception as e:
            print(f"Error submitting sell order: {e}")

# This function would be run on a schedule (e.g., once per day after market close)
if __name__ == '__main__':
    try:
        reconcile_position()
    except Exception as e:
        print(f"Error in main execution: {e}")

C:\Users\USER\anaconda3\envs\quant_env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


ModuleNotFoundError: No module named 'alpha_vantage.sectorperformance'